# Parámetros

In [6]:
BASE_DIR = '/Users/efrain.flores/Desktop/EF/Corner/Brands/New_SoM'
CPG_NAME = 'P&G'
CHANGES_FILE = 'Cambios Abr-May22'

# Código

In [7]:
from pathlib import Path

from pandas import read_csv

class CatalogChanges:
    def __init__(self, base_dir: str, cpg_name: str, changes_file: str) -> None:
        self.base_dir = Path(base_dir)
        self.cpg_name = cpg_name
        self.changes_file = changes_file
        self.catalog_path = self.base_dir.joinpath(self.cpg_name, f'{self.cpg_name}_catalog.csv')
        self.changes_path = self.base_dir.joinpath('Cambios', f'{self.changes_file}.csv')
        try: self.catalog = read_csv(self.catalog_path, sep='\t', encoding='utf-16')
        except: self.catalog = read_csv(self.catalog_path)
        self.changes = read_csv(self.changes_path)


    def merge_changes(self, old_category: list, new_category: list, to_drop: list, brand_id_col: str,**kwargs) -> None:
        self.catalog_columns = list(self.catalog.columns)
        self.df = self.catalog.merge(self.changes, how='left', **kwargs)

        for swap in zip(old_category, new_category):
            to_fill = []
            for old, new in zip(self.df[swap[0]], self.df[swap[-1]]):
                if str(new)=='nan': to_fill.append(old)
                else: to_fill.append(new)
            self.df[swap[-1]] = to_fill

        self.df.drop(to_drop, axis=1, inplace=True)
        self.df.drop(old_category, axis=1, inplace=True)
        self.df.rename(dict(zip(new_category, old_category)), axis=1, inplace=True)
        self.df = self.df[self.catalog_columns].copy()
        self.df.drop_duplicates(subset=old_category.extend(brand_id_col), inplace=True)

# Clase

In [8]:
cc = CatalogChanges(BASE_DIR, CPG_NAME, CHANGES_FILE)

# Transformar

In [9]:
cc.merge_changes(
    old_category=['category_id', 'category_en'], 
    new_category=['now', 'category_name'], 
    to_drop=['before', 'parent1_name'],
    brand_id_col='brand_id', 
    left_on='category_id', 
    right_on='before'
)
cc.df

,CPG_real,CPG,category_id,brand_id,brand_name,category_en,category_CPG
0,Smart,Competencia,974,10099,Amore,Feminine Care,Feminine Care
1,Sin CPG,Competencia,974,101376,CUTTIECUP,Feminine Care,Feminine Care
2,Sin CPG,Competencia,974,10525,MARCAS DE RENOM,Feminine Care,Feminine Care
3,Sin CPG,Competencia,974,111007,Soriana,Feminine Care,Feminine Care
4,Sin CPG,Competencia,974,111707,Pro Selection,Feminine Care,Feminine Care
...,...,...,...,...,...,...,...
1028,P&G MX,P&G,984,5852,Salvo,Laundry Detergent,Detergentes
1029,Genomma Lab,Competencia,358,16715,Genomma Lab,Stomach & saline solutions,CHD
1030,"Imalinx, Kirkland",Competencia,281,16792,Kirkland Signature,"Cold, flu & allergies",CHD
1031,Unilever,Competencia,1500,282,Nexxus,Haircare,Hair Care


# Exportar

In [10]:
cc.df.to_csv(cc.catalog_path.parent.joinpath(f'{cc.cpg_name}_catalog_update.csv'), index=False, sep='\t', encoding='utf-16')